# --- Part Two ---
As you try to work out what might be wrong, the reindeer tugs on your shirt and leads you to a nearby control panel. There, a collection of buttons lets you align the contraption so that the beam enters from any edge tile and heading away from that edge. (You can choose either of two directions for the beam if it starts on a corner; for instance, if the beam starts in the bottom-right corner, it can start heading either left or upward.)

So, the beam could start on any tile in the top row (heading downward), any tile in the bottom row (heading upward), any tile in the leftmost column (heading right), or any tile in the rightmost column (heading left). To produce lava, you need to find the configuration that energizes as many tiles as possible.

In the above example, this can be achieved by starting the beam in the fourth tile from the left in the top row:
```
.|<2<\....
|v-v\^....
.v.v.|->>>
.v.v.v^.|.
.v.v.v^...
.v.v.v^..\
.v.v/2\\..
<-2-/vv|..
.|<<<2-|.\
.v//.|.v..
```
Using this configuration, 51 tiles are energized:
```
.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..
```
Find the initial beam configuration that energizes the largest number of tiles; **how many tiles are energized in that configuration?**

In [1]:
from utilities import get_lines
import pandas as pd
from collections import namedtuple

In [2]:
layout = pd.DataFrame([[ch for ch in line] for line in get_lines('input',16)])

In [3]:
r,c = layout.shape

In [4]:
r,c = layout.shape
def reset_energy():
    return pd.DataFrame(0,index=pd.RangeIndex(r),columns=pd.RangeIndex(c))
energy = reset_energy()

def reset_splitters():
    return pd.DataFrame(0,index=pd.RangeIndex(r),columns=pd.RangeIndex(c))

splitters = reset_splitters()

In [5]:
Beam = namedtuple('Beam', 'row col dir')

In [6]:
def move_down(beam):
    row = beam.row+1
    return Beam(row, beam.col, 'S')

def move_up(beam):
    row = beam.row-1
    return Beam(row, beam.col, 'N')

def move_left(beam):
    col = beam.col-1
    return Beam(beam.row, col, 'W')

def move_right(beam):
    col = beam.col+1
    return Beam(beam.row, col, 'E')

In [7]:
def update_energy(energy, beam):
    if (0<=beam.row<r)&(0<=beam.col<c):
        val = energy.loc[beam.row,beam.col]
        energy.at[beam.row, beam.col] = 8 #  += 1 Don't need to keep adding values
        return True
    return False

In [8]:
def get_action(beam):
    return layout.loc[beam.row,beam.col]

In [9]:
keep_going = {'E':move_right,
              'W':move_left,
              'N':move_up,
              'S':move_down
             }

back_mirror = {'E':move_down,
               'W':move_up,
               'N':move_left,
               'S':move_right
              }

forward_mirror = {'E':move_up,
                  'W':move_down,
                  'N':move_right,
                  'S':move_left
                 }

In [10]:
def get_splitter_state(beam):
    r,c = beam.row, beam.col
    return splitters.loc[r,c]

In [11]:
def get_180_dir(dir):
    compass = 'NESW'
    i = compass.index(dir)
    return compass[(i+2)%4]

In [12]:
def get_90_dir(dir):
    compass = 'NESW'
    i = compass.index(dir)
    return compass[(i+1)%4]

In [13]:
def mark_used_splitter(beam):
    splitters.loc[beam.row, beam.col]=1 #  mark splitter as used
    return None

In [14]:
def split_beam(beam):
    dirs = 'NESW'
    opp_dir = get_180_dir(beam.dir)
    new_beam = Beam(beam.row, beam.col, opp_dir)
    # print(beam, 'and new beam', new_beam)
    return new_beam

In [15]:
def split_vert(beam): #  | splitter
    if (beam.dir=='N')|(beam.dir=='S'):
        return keep_going[beam.dir](beam)
    else: # splitter actions are to mark splitters layout and then either create a new beam or stop
        if get_splitter_state(beam)==0:
            beam = Beam(beam.row, beam.col, get_90_dir(beam.dir)) # make a right turn
            beams.append(split_beam(beam)) #  create a new beam 
            mark_used_splitter(beam) #  mark the splitter as used
            return keep_going[beam.dir](beam)
        else:
            return Beam(-1,-1,beam.dir) #  beam is off the layout...will stop
            

In [16]:
def split_horiz(beam): #  - splitter
    if (beam.dir=='E')|(beam.dir=='W'):
        return keep_going[beam.dir](beam)
    else: # splitter actions are to mark splitters layout and then either create a new beam or stop
        if splitters.loc[beam.row, beam.col]==0:
            beam = Beam(beam.row, beam.col, get_90_dir(beam.dir))
            beams.append(split_beam(beam)) #  create a new beam
            mark_used_splitter(beam) #  mark the splitter as used
            return keep_going[beam.dir](beam)
        else:
            return Beam(-1,-1,beam.dir) #  force stop, splitter has already been used

In [17]:
def take_step(beam):
    action = get_action(beam)
    if action=='/': #  forward slash mirror, nicknamed forward mirror
        beam = forward_mirror[beam.dir](beam)
    elif action=='\\': #  back slash mirror, nicknamed back mirror
        beam = back_mirror[beam.dir](beam)
    elif action=='.':
        beam = keep_going[beam.dir](beam)
    elif action=='|':
        beam = split_vert(beam)
    elif action=='-':
        beam = split_horiz(beam)
    return beam

In [18]:
def activate_beam(beam):
    on_layout = update_energy(energy, beam)
    while on_layout:
        beam = take_step(beam)
        on_layout = update_energy(energy, beam)
    

In [19]:
beams_from_top = [Beam(0,col,'S') for col in range(c)]
beams_from_right = [Beam(row,c-1,'W') for row in range(r)]
beams_from_bottom = [Beam(r-1,col,'N') for col in range(c)]
beams_from_left = [Beam(row,0,'E') for row in range(r)]
beams_from_top.extend(beams_from_right)
beams_from_top.extend(beams_from_bottom)
beams_from_top.extend(beams_from_left)
starting_beams = beams_from_top
len(starting_beams)

440

In [20]:
def get_total_energy(energy):
    total = 0
    for i in range(r):
        for j in range(c):
            if energy.loc[i,j]!=0:
                total+=1
    return total

In [26]:
max_total_energy = 0
max_beam = Beam(-1,-1,'W')
counter = 0
for beam in starting_beams:
    beams = [beam]
    energy = reset_energy()
    splitters = reset_splitters()
    while len(beams)>0:
        beam = beams.pop()
        activate_beam(beam)
    total_energy = get_total_energy(energy)
    if total_energy>max_total_energy:
        max_total_energy = total_energy
    counter += 1
    if counter%10==0:
        print(f'{counter} complete',end='\t')
    
print(f'Maximum energy {max_total_energy}')

10 complete	20 complete	30 complete	40 complete	50 complete	60 complete	70 complete	80 complete	90 complete	100 complete	110 complete	120 complete	130 complete	140 complete	150 complete	160 complete	170 complete	180 complete	190 complete	200 complete	210 complete	220 complete	230 complete	240 complete	250 complete	260 complete	270 complete	280 complete	290 complete	300 complete	310 complete	320 complete	330 complete	340 complete	350 complete	360 complete	370 complete	380 complete	390 complete	400 complete	410 complete	420 complete	430 complete	440 complete	Maximum energy 7324
